# SVD та LU Декомпозиція: Продакшн Приклади

## Наукові джерела

### SVD (Singular Value Decomposition):
1. **Zhang, Z. (2015).** "The Singular Value Decomposition, Applications and Beyond" - arXiv:1510.08532
   - Застосування в machine learning, matrix completion, і data classification

2. **Sarwar et al. (2000, 2002).** Рекомендаційні системи на основі SVD для MovieLens dataset
   - Використання SVD для collaborative filtering

3. **Deerwester et al. (1990), Berry et al. (1995).** Latent Semantic Analysis (LSA)
   - SVD для аналізу текстів та пошуку документів

4. **Henry et al. (2002).** "Application of SVD to Macromolecular X-Ray Data" - PMC1302779
   - Застосування в кристалографії та біології

### LU Декомпозиція:
1. **Bartels & Golub (1969).** "The simplex method using LU decomposition" - Communications of the ACM
   - Застосування в linear programming

2. **Toloo et al. (2015).** "LU Decomposition in DEA" - Computational Economics
   - Використання для Data Envelopment Analysis в аналізі ефективності лікарень

3. **Stabrowski (2018).** "Parallel LU decomposition: Gauss vs. Crout algorithm"
   - Реальні застосування з Florida University Matrix Collection

---

In [ ]:
# Імпорт необхідних бібліотек
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg
from PIL import Image
import requests
from io import BytesIO

print("✅ Бібліотеки завантажено")

# Частина 1: SVD - Рекомендаційна Система (Production)

## Приклад: Netflix-стиль рекомендацій фільмів

**Джерело:** Sarwar et al. (2000), "Application of Dimensionality Reduction in Recommender System"

**Продакшн використання:**
- Netflix, Amazon Prime, Spotify
- Обробка мільйонів користувачів та тисяч фільмів
- Зменшення розмірності з 10,000+ фільмів до 50-200 латентних факторів

In [ ]:
# Створюємо матрицю оцінок користувачів (users x movies)
# 0 означає "не оцінено"
ratings = np.array([
    # Термінатор, Аватар, Ноутбук, Титанік, Месники, Інтерстеллар
    [5, 5, 1, 1, 5, 5],  # Андрій (любить екшн/фантастику)
    [1, 2, 5, 5, 1, 2],  # Марія (любить романтику)
    [5, 4, 2, 1, 5, 4],  # Петро (любить екшн)
    [2, 3, 4, 5, 2, 3],  # Оля (любить драми)
    [4, 5, 1, 2, 4, 5],  # Іван (любить sci-fi)
    [1, 1, 5, 4, 2, 2],  # Ганна (любить мелодрами)
], dtype=float)

movie_names = ['Термінатор', 'Аватар', 'Ноутбук', 'Титанік', 'Месники', 'Інтерстеллар']
user_names = ['Андрій', 'Марія', 'Петро', 'Оля', 'Іван', 'Ганна']

print("Матриця оцінок (Users x Movies):")
print("\n".join([f"{user:8} {list(ratings[i])}" for i, user in enumerate(user_names)]))
print(f"\nРозмір: {ratings.shape[0]} користувачів × {ratings.shape[1]} фільмів")

In [ ]:
# Виконуємо SVD
U, sigma, VT = np.linalg.svd(ratings, full_matrices=False)

print("SVD Декомпозиція:")
print(f"U (користувачі → жанри): {U.shape}")
print(f"Σ (важливість жанрів): {sigma.shape}")
print(f"V^T (фільми → жанри): {VT.shape}")
print(f"\nСингулярні значення (важливість факторів): {sigma}")

In [ ]:
# Візуалізація: скільки компонент залишити?
variance_explained = (sigma**2) / np.sum(sigma**2)
cumulative_variance = np.cumsum(variance_explained)

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.bar(range(1, len(sigma)+1), variance_explained, alpha=0.7)
plt.xlabel('Компонента')
plt.ylabel('% поясненої варіації')
plt.title('Важливість кожної компоненти')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(range(1, len(sigma)+1), cumulative_variance, 'o-', linewidth=2)
plt.axhline(y=0.9, color='r', linestyle='--', label='90% порог')
plt.xlabel('Кількість компонент')
plt.ylabel('Кумулятивна % варіації')
plt.title('Скільки компонент залишити?')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Для продакшну: зберігаємо лише топ-k компонент
k = 2  # Netflix зазвичай використовує 50-200 факторів
print(f"\n✅ Використовуючи {k} компоненти, ми зберігаємо {cumulative_variance[k-1]:.1%} інформації")

In [ ]:
# Compressed representation (Product в продакшні)
U_k = U[:, :k]
sigma_k = np.diag(sigma[:k])
VT_k = VT[:k, :]

# Рекомендація: Які фільми порадити Андрію?
user_idx = 0  # Андрій
user_vector = U_k[user_idx] @ sigma_k  # Профіль Андрія в "жанровому" просторі

# Обчислюємо схожість з усіма фільмами
movie_vectors = VT_k.T  # Транспонуємо щоб отримати (movies x latent_factors)
similarities = movie_vectors @ user_vector.T

print(f"\n🎬 Рекомендації для {user_names[user_idx]}:")
recommendations = sorted(zip(movie_names, similarities, ratings[user_idx]), 
                        key=lambda x: x[1], reverse=True)

for movie, score, original_rating in recommendations:
    status = "✓ вже подивився" if original_rating > 3 else "← РЕКОМЕНДУЄМО"
    print(f"  {movie:15} (score: {score:5.2f}) {status}")

In [ ]:
# PRODUCTION BONUS: Економія пам'яті
original_size = ratings.size * 8  # 8 bytes per float64
compressed_size = (U_k.size + sigma_k.size + VT_k.size) * 8
compression_ratio = original_size / compressed_size

print(f"\n💾 Економія пам'яті:")
print(f"  Оригінал: {original_size} bytes")
print(f"  Compressed (k={k}): {compressed_size} bytes")
print(f"  Compression ratio: {compression_ratio:.2f}x")
print(f"\n  У реальному Netflix (100M users × 10K movies):")
real_original = 100_000_000 * 10_000 * 8 / (1024**3)
real_compressed = (100_000_000 * 200 + 10_000 * 200) * 8 / (1024**3)
print(f"    Оригінал: {real_original:.1f} GB")
print(f"    SVD (k=200): {real_compressed:.1f} GB")
print(f"    Економія: {real_original/real_compressed:.1f}x")

# Частина 2: LU Декомпозиція - Фінансове Моделювання (Production)

## Приклад: Оптимізація портфелю інвестицій

**Джерело:** Bartels & Golub (1969), "Simplex method using LU decomposition"

**Продакшн використання:**
- Фінансові інститути (Goldman Sachs, BlackRock)
- Розв'язування тисяч систем рівнянь щодня
- Перерахунок портфелів при зміні ринкових умов

In [ ]:
# Задача: Розподіл $1,000,000 між різними активами
# Обмеження:
#   - Мінімум 20% у облігаціях (низький ризик)
#   - Максимум 40% у акціях (високий ризик)
#   - Решта в нерухомість
#   - Цільова прибутковість: 8%

# Система рівнянь:
# x + y + z = 1,000,000  (весь капітал)
# 0.05x + 0.12y + 0.08z = 80,000  (8% прибутковість)
# x = 200,000  (20% в облігаціях)

A = np.array([
    [1.0, 1.0, 1.0],      # Загальна сума
    [0.05, 0.12, 0.08],   # Прибутковість
    [1.0, 0.0, 0.0]       # Облігації = 200k
])

b = np.array([1_000_000, 80_000, 200_000])

print("Система рівнянь (Ax = b):")
print("A:")
print(A)
print(f"\nb: {b}")

In [ ]:
# LU Декомпозиція
P, L, U = linalg.lu(A)

print("\nLU Декомпозиція:")
print("\nP (Permutation matrix):")
print(P)
print("\nL (Lower triangular):")
print(L)
print("\nU (Upper triangular):")
print(U)
print("\n✅ Перевірка: A = PLU")
print(np.allclose(A, P @ L @ U))

In [ ]:
# Розв'язуємо систему в два кроки
# Крок 1: Forward substitution (Ly = Pb)
Pb = P @ b
y = linalg.solve_triangular(L, Pb, lower=True)
print("Крок 1 - Forward substitution (Ly = Pb):")
print(f"y = {y}")

# Крок 2: Backward substitution (Ux = y)
x = linalg.solve_triangular(U, y, lower=False)
print("\nКрок 2 - Backward substitution (Ux = y):")
print(f"x = {x}")

print("\n💰 Розподіл інвестицій:")
print(f"  Облігації (5% прибутковість):  ${x[0]:,.0f} ({x[0]/1_000_000*100:.1f}%)")
print(f"  Акції (12% прибутковість):     ${x[1]:,.0f} ({x[1]/1_000_000*100:.1f}%)")
print(f"  Нерухомість (8% прибутковість): ${x[2]:,.0f} ({x[2]/1_000_000*100:.1f}%)")
print(f"\n  Очікуваний дохід: ${(0.05*x[0] + 0.12*x[1] + 0.08*x[2]):,.0f}")

In [ ]:
# PRODUCTION SCENARIO: Зміна ринкових умов
# Припустимо, прибутковість акцій змінилася з 12% на 10%
# LU вже обчислено, просто змінюємо b!

print("\n🔄 PRODUCTION BONUS: Швидкий перерахунок при зміні умов\n")

import time

# Нові цільові прибутковості
new_targets = [70_000, 75_000, 80_000, 85_000, 90_000]

print("Метод 1: Повне розв'язування (без LU):")
start = time.time()
for target in new_targets:
    b_new = np.array([1_000_000, target, 200_000])
    x_new = np.linalg.solve(A, b_new)
time_without_lu = time.time() - start
print(f"  Час: {time_without_lu*1000:.3f} ms")

print("\nМетод 2: Використання існуючого LU:")
start = time.time()
for target in new_targets:
    b_new = np.array([1_000_000, target, 200_000])
    Pb = P @ b_new
    y = linalg.solve_triangular(L, Pb, lower=True)
    x_new = linalg.solve_triangular(U, y, lower=False)
time_with_lu = time.time() - start
print(f"  Час: {time_with_lu*1000:.3f} ms")

print(f"\n✅ Прискорення: {time_without_lu/time_with_lu:.2f}x")
print(f"\n  У продакшні (1000 перерахунків на день):")
print(f"    Без LU: {time_without_lu*1000*1000:.0f} ms = {time_without_lu*1000:.2f} секунд")
print(f"    З LU:   {time_with_lu*1000*1000:.0f} ms = {time_with_lu*1000:.2f} секунд")
print(f"    Економія часу: {(time_without_lu-time_with_lu)*1000:.2f} секунд/день")

# Порівняння SVD vs LU

| Критерій | SVD | LU |
|----------|-----|----|
| **Застосування** | Рекомендації, стиснення, ML | Розв'язування систем, оптимізація |
| **Складність** | O(mn²) | O(n³) |
| **Коли використовувати** | Аналіз даних, зменшення розмірності | Багаторазові розв'язування |
| **Продакшн приклади** | Netflix, Spotify, Amazon | Фінанси, інженерія, моделювання |
| **Головна перевага** | Знаходить приховані патерни | Швидке повторне розв'язування |

## Висновки

**SVD:**
- ✅ Ідеально для великих розріджених матриць (sparse matrices)
- ✅ Знаходить латентні фактори в даних
- ✅ Дозволяє компресію з мінімальною втратою точності
- 📊 Використовується у: рекомендаційних системах, обробці зображень, NLP

**LU:**
- ✅ Дуже ефективно для багаторазових розв'язувань
- ✅ Числово стабільніше за прямі методи
- ✅ Легко паралелізується
- 📊 Використовується у: фінансовому моделюванні, інженерних симуляціях, оптимізації

## Додаткові ресурси

1. **SVD Tutorial:** https://arxiv.org/abs/1510.08532
2. **Netflix Prize:** https://www.netflixprize.com/
3. **LAPACK Documentation:** http://www.netlib.org/lapack/
4. **Numerical Recipes:** Chapter 2.3 (LU) and 2.6 (SVD)